# 1. Полиморфизм

Свойство кода работать с разными типами данных называют **полиморфизмом**.

Мы уже неоднократно пользовались этим свойством многих функций и операторов, не задумываясь о нем. Например, оператор + является полиморфным:

In [1]:
print(1 + 2)          # 3
print(1.5 + 0.2)      # 1.7
print("abc" + "def")  # abcdef

3
1.7
abcdef


Внутренняя реализация оператора + существенно отличается для целых чисел, чисел с плавающей точкой и строк. То есть на самом деле это три разные операции — интерпретатор Python выбирает одну из них при выполнении в зависимости от операндов. Впрочем, в нашем случае выбор очевиден, потому что операнды — просто константы.

In [2]:
from math import pi

 
class Circle:
    def __init__(self, radius):
        self.radius = radius
 
    def area(self):
        return pi * self.radius ** 2
 
    def perimeter(self):
        return 2 * pi * self.radius
 

class Square:
    def __init__(self, side):
        self.side = side
 
    def area(self):
        return self.side * self.side
 
    def perimeter(self):
        return 4 * self.side

In [3]:
def print_shape_info(shape):
    print(f"Area = {shape.area()}, perimeter = {shape.perimeter()}.")

In [4]:
square = Square(10)
print_shape_info(square)

Area = 100, perimeter = 40.


In [5]:
circle = Circle(10)
print_shape_info(circle)

Area = 314.1592653589793, perimeter = 62.83185307179586.


В данном примере мы имеем функцию **print_shape_info**, которая предполагает наличие у передаваемого ей аргумента методов **area** и **perimeter**. Причем не важно, к каким классам(типам) относятся данные объекты.

**Утиная типизация**

Данный код использует тот факт, что в Python принята так называемая утиная типизация. Название происходит от шутливого выражения «Если нечто выглядит как утка, плавает как утка и крякает как утка, это, вероятно, утка и есть».

В программах на Python это означает, что если какой-то объект поддерживает все требуемые от него операции, с ним и будут работать с помощью этих операций, не заботясь о том, какого он на самом деле типа.

Чтобы полиморфизм работал, за ними надо следить как на уровне синтаксиса (одинаковые имена методов и количество параметров), так и на уровне смысла (методы с одинаковыми именами делают похожие операции, параметры методов имеют тот же смысл).

In [6]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height
 
    def area(self):
        return self.width * self.height
 
    def perimeter(self):
        return 2 * (self.width + self.height)
 

rect = Rectangle(10, 15)
print_shape_info(rect)  # Area = 150, perimeter = 50.

Area = 150, perimeter = 50.


Еще раз обратите внимание: утиная типизация позволяет заранее написать функцию, которая будет работать со всеми экземплярами любых классов — даже еще не существующих. Важно лишь, чтобы эти классы поддерживали необходимый функции **интерфейс**.

## 1.1. Функция **isinstance**

Эта функция принимает два параметра: isinstance(object, type)

Первый параметр представляет объект, а второй — тип, на принадлежность к которому выполняется проверка. Если объект представляет указанный тип, функция возвращает True.

In [12]:
class Student:
    def __init__(self, name, university):
        self.university = university
        self.name = name
        
class Employee:
    def __init__(self, name, company):
        self.company = company
        self.name = name
        
class Pupil:
    def __init__(self, name, school):
        self.school = school
        self.name = name
        
p1 = Pupil('Ivanov', '1')
p2 = Employee('Petrov', 'MTS')
p3 = Student('Sidorov', 'Kbsu')
people = [p1, p2, p3]

for person in people:
    if isinstance(person, Student):
        print(person.university)
    elif isinstance(person, Employee):
        print(person.company)
    else:
        print(person.name)
    print()

Ivanov

MTS

Kbsu



Здесь мы имеем дело со списком объектов, порожденных разными классами, при при этом перебор элементов содержит проверку принадлежности одному из классов.

In [13]:
isinstance(p1, Pupil)

True

In [14]:
type(p1)

__main__.Pupil

# 2. Специальные методы

Специальные методы имеют для интерпретатора особое значение. Имена специальных методов и их смысл определены создателями языка: создавать новые нельзя, можно только реализовывать существующие. Названия всех специальных методов начинаются и заканчиваются на два подчеркивания.

Пример такого метода — уже знакомый нам `__init__`. Он предназначен для инициализации экземпляров и автоматически вызывается интерпретатором после создания экземпляра объекта.

## 2.1. `__add__` и `__iadd__`

In [15]:
class Time:
    def __init__(self, minutes, seconds):
        self.minutes = minutes
        self.seconds = seconds

    def __add__(self, other):
        m = self.minutes + other.minutes
        s = self.seconds + other.seconds
        m += s // 60
        s = s % 60
        return Time(m, s)

    def __iadd__(self, other):
        m = self.minutes + other.minutes
        s = self.seconds + other.seconds
        m += s // 60
        s = s % 60
        self.minutes = m
        self.seconds = s
        return self

    def info(self):
        return f'{self.minutes}:{self.seconds}'


t1 = Time(5, 50)
print(t1.info())  # 5:50
t2 = Time(3, 20)
print(t2.info())  # 3:20
print(id(t1))
t1 += t2
print(t1.info())  # 9:10
print(id(t1))  # id объекта не поменяется

5:50
3:20
9:10
140442484012848 140442090814240 140442090814672


В данном примере определены 2 операции для класса Time - операции сложения - `+` и `+=`. Первая возвращает новый объект того же типа, вторая - ссылку на измененный исходный объект.

Здесь нужно обратить внимание на аргумент **`other`**, который в данном случае будет содержать "то, с чем мы складываем наш объект".

## 2.2. `__repr__` и `__str__`

методы необходимы для текстового представления объектов. Первый обычно используется для возвращения строки кода, инициализирующую объект, второй - для детализации вывода при печати объекта

In [25]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f'Человек по имени {self.name}'
    
    def __repr__(self):
        return f'Person(\'{self.name}\')'

In [26]:
p1 = Person('Иванов')
print(p1)

Человек по имени Иванов


In [27]:
print(repr(p1))

Person('Иванов')


## 2.3. `__sub__`, `__mul__`, `__rmul__`

In [29]:
class MyVector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
 
    def __add__(self, other):
        return MyVector(self.x + other.x, self.y + other.y)
 
    def __sub__(self, other):
        return MyVector(self.x - other.x, self.y - other.y)
 
    def __mul__(self, other):
        return MyVector(self.x * other, self.y * other)
 
    def __rmul__(self, other):
        return MyVector(self.x * other, self.y * other)
 
    def __str__(self):
        return f'MyVector({self.x}, {self.y})'
 
 
v1 = MyVector(-2, 5)
v2 = MyVector(3, -4)
v_sum = v1 + v2
print(v_sum)
v_mul = v1 * 1.5
print(v_mul)
v_rmul = -2 * v1
print(v_rmul)  

MyVector(1, 1)
MyVector(-3.0, 7.5)
MyVector(4, -10)


## 2.4. Другие операторы


Метод	                                   Описание  
`__add__`(self, other)	           Сложение (x + y). Будет вызвано: x.`__add__`(y)  
`__sub__`(self, other)	           Вычитание (x - y)  
`__mul__`(self, other)	           Умножение (x * y)  
`__truediv__`(self, other)	       Деление (x / y)  
`__floordiv__`(self, other)	       Целочисленное деление (x // y)  
`__mod__`(self, other)	           Остаток от деления (x % y)  
`__divmod__`(self, other)	           Частное и остаток (divmod(x, y))  
`__radd__`(self, other)	           Сложение (x + y). Будет вызвано: y.`__radd__`(x)  
`__rsub__`(self, other)	           Вычитание (y - x)  
`__lt__`(self, other)	               Сравнение (x < y). Будет вызвано: x.`__lt__`(y)  
`__eq__`(self, other)	               Сравнение (x == y). Будет вызвано: x.`__eq__`(y)  
`__len__`(self)	                   Возвращение длины объекта  
`__getitem__`(self, key)	           Доступ по индексу (или ключу)  
`__call__`(self[, args...])	       Вызов экземпляра класса как функции  

Это лишь небольшая часть методов. Остальные можно посмотреть в документации:
https://docs.python.org/3/reference/datamodel.html#special-method-names

Однако есть некоторые общие правила, например для арифметических операций (`__add__`, `__mul__` и т.д.) добавление в название символа `i` означает сокращенную форму операции (`+ => +=`), а символа `r` - обратный порядок вычисления.

# 3. Наследование

Одно из главных достоинств ООП заключается в многократном использовании одного и того же кода. Один из способов достижения этого - механизм `наследования`. Легче всего представить себе наследование в виде отношения между классами как `тип` и `подтип`. 

In [41]:
class SchoolMember:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        print(f'Создан SchoolMember: {self.name}')
    
    def tell(self):
        print(f'Имя:{self.name} Возраст:{self.age}')

class Teacher(SchoolMember):
    def __init__(self, name, age, salary):
        SchoolMember.__init__(self, name, age)
        self.salary = salary
        print(f'Создан Teacher: {self.name}')

    def info(self):
        print(f'Зарплата {self.salary}')

class Student(SchoolMember):
    def __init__(self, name, age, marks):
        super().__init__(name, age)
        self.marks = marks
        print(f'Создан Student: {self.name}')
    
    def tell(self):
        super().tell()
        print(f'Оценки: {self.marks}')
        
        
t = Teacher('mr. S. Ivanov', 40, 50_000)
s = Student('V. Pupkin', 25, 75)
    

Создан SchoolMember: mr. S. Ivanov
Создан Teacher: mr. S. Ivanov
Создан SchoolMember: V. Pupkin
Создан Student: V. Pupkin


Здесь первые строки методов `__init__` классов `Teacher` и `Student` имеют идентичный смысл: поскольку каждый из этих классов является потомком класса `SchoolMember` мы обязаны вызвать его метод инициализации (еще его называют конструктором), передав ему необходимые параметры.   

In [36]:
t.tell()

Имя:mr. S. Ivanov Возраст:40
Зарплата 50000


In [40]:
s.tell()

Имя:V. Pupkin Возраст:25


Поскольку оа класса `Teacher` и `Student` имеют однин базовый класс, им доступны методы tell этого класса. Но в случае со `Student` - в нем данный метод переопределен(заменен).

Рассмотрим еще 1 пример:

In [42]:
class Shape:
    def describe(self):
        # Атрибут __class__ содержит класс или тип объекта self
        # Атрибут __name__ содержит строку, 
        # в которой написано название класса или типа
        print(f"Класс: {self.__class__.__name__}")

In [43]:
from math import pi


class Circle(Shape):
    def __init__(self, radius):
        self.r = radius

    def area(self):
        return pi * self.r ** 2
    
    def perimeter(self):
        return 2 * pi * self.r
    
    def square(self):
        side = pi ** 0.5 * self.r
        return Rectangle(side, side)

class Rectangle(Shape):
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def area(self):
        return self.a * self.b
    
    def perimeter(self):
        return 2 * (self.a + self.b)


shape = Shape()
shape.describe()

circle = Circle(1)
circle.describe()

rectangle = Rectangle(1, 2)
rectangle.describe()

Класс: Shape
Класс: Circle
Класс: Rectangle


In [44]:
class Circle(Shape):
    def __init__(self, radius):
        self.r = radius

    def area(self):
        return pi * self.r ** 2


circle = Circle(1)
square = circle.square()

print(f"Площадь круга:    {circle.area()}")
print(f"Площадь квадрата: {square.area()}")
print(f"Радиус круга: {circle.r}")
print(f"Длина стороны квадрата: {square.a}")

Площадь круга:    3.141592653589793
Площадь квадрата: 3.1415926535897927
Радиус круга: 1
Длина стороны квадрата: 1.7724538509055159


In [ ]:
class Square(Rectangle):
    def __init__(self, size):
        print('Создаем квадрат')
        super().__init__(size, size)


side = 5
sq = Square(side, side)
print(sq.area())
print(sq.perimeter())


In [ ]:
sq = Square(2)
print(sq.area())
print(sq.perimeter())
print(sq.a)

## 3.1. Множественное наследование

Python поддерживает множественное наследование - наследование от 2х и более классов.

<img src="https://pythonim.ru/wp-content/uploads/img-70.png">



In [2]:
class Person: 
    def __init__(self, personName, personAge): 
        self.name = personName 
        self.age = personAge 
    
    def showName(self): 
        print(self.name) 
        
    def showAge(self): 
        print(self.age) 

class Student: 
    def __init__(self, studentId): 
        self.studentId = studentId 
        
    def getId(self): 
        return self.studentId 
    
class Resident(Person, Student): # extends both Person and Student class 
    def __init__(self, name, age, id): 
        Person.__init__(self, name, age) 
        Student.__init__(self, id) 

resident1 = Resident('John', 30, '102') 
resident1.showName() 
print(resident1.getId())

John
102


В этом примере класс Resident получается в результате наследования от классов Person и Student

In [4]:
class A: 
    def __init__(self): 
        self.name = 'John' 
        self.age = 23 
    
    def getName(self): 
        return self.name 

class B: 
    def __init__(self): 
        self.name = 'Richard' 
        self.id = '32' 
    
    def getName(self): 
        return self.name 

class C(A, B): 
    def __init__(self): 
        A.__init__(self) 
        B.__init__(self) 
    
    def getName(self): 
        return self.name 

C1 = C() 
print(C1.getName())

Richard


In [5]:
class A: 
    def __init__(self): 
        super().__init__() 
        self.name = 'John' 
        self.age = 23 
    
    def getName(self): 
        return self.name 

class B: 
    def __init__(self): 
        super().__init__() 
        self.name = 'Richard' 
        self.id = '32' 
    
    def getName(self): 
        return self.name 

class C(A, B): 
    def __init__(self): 
        super().__init__()
    
    def getName(self): 
        return self.name 

C1 = C() 
print(C1.getName())


John


Здесь super() в конструкторе класса С вызывает оба конструктора суперклассов А и В, и в случае конфликтов (у обоих этих классов есть метод getName) приоритет определяется порядком внутри скобок, следующих за определением целевого класса: class C(A,B).